**Sistemas Difusos**

* _Maria Alejandra Bonilla Diaz - 20251595002_ 
* _Alvaro Alejandro Zarabanda Gutierrez – 20251595006_


$x = \\frac{ -b \\pm \\sqrt{ b^2 - 4ac } }{ 2a }$


In [1]:
print("Hola mundo");

Hola mundo


$x = \\\\frac{ -b \\\\pm \\\\sqrt{ b^2 - 4ac } }{ 2a }$